In [2]:
from src import scrapers
test_scraper = scrapers.PfrScraper(
    '/Users/Jared/OneDrive - Sportsbook Science/data/raw/nfl')

In [5]:
test_scraper.selenium_delay = 3.5
game_summaries_df = test_scraper.get_game_summaries(seasons=2021, weeks='all')
game_summaries_df.to_csv('data/2021.csv', index=False)

--- getting season 2021 ---
-- getting week: 1 --
- loaded saved game: /boxscores/202109090tam.htm -
- loaded saved game: /boxscores/202109120atl.htm -
- loaded saved game: /boxscores/202109120buf.htm -
- loaded saved game: /boxscores/202109120car.htm -
- loaded saved game: /boxscores/202109120cin.htm -
- loaded saved game: /boxscores/202109120clt.htm -
- loaded saved game: /boxscores/202109120det.htm -
- loaded saved game: /boxscores/202109120htx.htm -
- loaded saved game: /boxscores/202109120oti.htm -
- loaded saved game: /boxscores/202109120was.htm -
- loaded saved game: /boxscores/202109120kan.htm -
- loaded saved game: /boxscores/202109120nor.htm -
- loaded saved game: /boxscores/202109120nwe.htm -
- loaded saved game: /boxscores/202109120nyg.htm -
- loaded saved game: /boxscores/202109120ram.htm -
- loaded saved game: /boxscores/202109130rai.htm -
-- getting week: 2 --
- loaded saved game: /boxscores/202109160was.htm -
- loaded saved game: /boxscores/202109190car.htm -
- loaded s

In [3]:
test_scraper.selenium_delay = 3.5
game_summaries_df = test_scraper.get_game_summaries(seasons=[1999,2009], weeks=[2,13])
game_summaries_df.to_csv('data/1999and2009_weeks2and13.csv', index=False)

--- getting season 1999 ---
-- getting week: 2 --
- loaded saved game: /boxscores/199909190cin.htm -
- loaded saved game: /boxscores/199909190min.htm -
- loaded saved game: /boxscores/199909190car.htm -
- loaded saved game: /boxscores/199909190det.htm -
- loaded saved game: /boxscores/199909190mia.htm -
- loaded saved game: /boxscores/199909190rav.htm -
- loaded saved game: /boxscores/199909190nwe.htm -
- loaded saved game: /boxscores/199909190chi.htm -
- loaded saved game: /boxscores/199909190nyg.htm -
- loaded saved game: /boxscores/199909190phi.htm -
- loaded saved game: /boxscores/199909190sfo.htm -
- loaded saved game: /boxscores/199909190oti.htm -
- loaded saved game: /boxscores/199909190kan.htm -
- loaded saved game: /boxscores/199909190buf.htm -
- loaded saved game: /boxscores/199909200dal.htm -
-- getting week: 13 --
- loaded saved game: /boxscores/199912020jax.htm -
- loaded saved game: /boxscores/199912050cin.htm -
- loaded saved game: /boxscores/199912050det.htm -
- loaded 